# 💊 Pharma Navigator - RAG System
Turkish Drug Prospectus Assistant with DSPy + FAISS + Chainlit

**Setup Instructions:**
1. Run cells in order
2. Copy ngrok token when asked
3. Access Chainlit UI via provided link

---

## Step 1: Clone & Setup Environment

In [ ]:
# Clone repository (replace with your repo URL)
%cd /content
!git clone https://github.com/YOUR-USERNAME/pharma-navigator.git 2>/dev/null || echo "Already cloned"
%cd /content/pharma-navigator
!pwd

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt
!pip install -q pyngrok  # For public URL

print("✅ Dependencies installed")
print(f"🐍 Python: {!python --version}")

## Step 2: Configure Environment

In [ ]:
import os
from google.colab import userdata

# Get Cerebras API key from Colab Secrets
# Go to 🔑 Secrets in left sidebar, add CEREBRAS_API_KEY
cerebras_key = userdata.get('CEREBRAS_API_KEY')

if cerebras_key:
    os.environ['CEREBRAS_API_KEY'] = cerebras_key
    print("✅ CEREBRAS_API_KEY configured")
else:
    print("⚠️ CEREBRAS_API_KEY not found")
    print("Add it in left sidebar: 🔑 Secrets > Add CEREBRAS_API_KEY")

## Step 3: Verify Database

In [ ]:
import os
from pathlib import Path

db_path = Path("/content/pharma-navigator/faiss_db")

if (db_path / "faiss.index").exists():
    print("✅ FAISS database found")
    # Show stats
    index_size = (db_path / "faiss.index").stat().st_size / (1024*1024)
    print(f"📊 Index size: {index_size:.2f} MB")
else:
    print("❌ FAISS database not found")
    print("📝 Upload PDF files and run ingestion script")

## Step 4: Start Chainlit Server with ngrok

In [ ]:
import subprocess
import time
from pyngrok import ngrok
import os

# Kill any existing processes
os.system("pkill -f chainlit 2>/dev/null || true")
time.sleep(2)

# Start Chainlit in background
print("🚀 Starting Chainlit server...")
subprocess.Popen(
    ["python3", "-m", "chainlit", "run", "src/app.py", "-h", "--host", "0.0.0.0", "--port", "8000"],
    cwd="/content/pharma-navigator",
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# Wait for server to start
time.sleep(10)

# Setup ngrok tunnel
print("🌐 Setting up ngrok tunnel...")
public_url = ngrok.connect(8000, "http")

print("\n" + "="*60)
print("✅ SERVER READY!")
print("="*60)
print(f"🔗 Public URL: {public_url}")
print("\n📝 Share this link to access the Chainlit UI")
print("="*60)

## Step 5: Test Backend (Optional)

In [ ]:
# Optional: Test backend directly
import sys
sys.path.insert(0, '/content/pharma-navigator')

from src.models.intent import classify_intent
from src.models.qa import generate_answer
from src.retrieval.retriever import DrugRetriever
import dspy
import tomli
import os

# Load config
with open('/content/pharma-navigator/config.toml', 'rb') as f:
    CONFIG = tomli.load(f)

# Initialize LM
lm = dspy.LM(
    model=f"cerebras/{CONFIG['llm']['model']}",
    api_key=os.getenv("CEREBRAS_API_KEY"),
    api_base="https://api.cerebras.ai/v1",
    temperature=CONFIG['llm']['temperature'],
    max_tokens=CONFIG['llm']['max_tokens']
)
dspy.configure(lm=lm)

# Load retriever
retriever = DrugRetriever(
    db_path=CONFIG['database']['path'],
    embedding_model=CONFIG['embedding']['model']
)

stats = retriever.get_collection_stats()
print(f"✅ Database: {stats['total_chunks']} chunks from {len(stats['unique_drugs'])} drugs")
print(f"📚 Drugs: {', '.join(stats['unique_drugs'])}")

# Test query
test_query = "Cipralex nasıl kullanılır?"
print(f"\n🧪 Test Query: {test_query}")

intent = classify_intent(test_query, lm)
print(f"✅ Intent: Drug-related = {intent['is_drug_related']}")

if intent['is_drug_related']:
    retrieval = retriever.retrieve(
        query=test_query,
        drug_names=intent['drug_names'] if intent['drug_names'] else None,
        top_k=CONFIG['retrieval']['top_k'],
        similarity_threshold=CONFIG['retrieval']['similarity_threshold']
    )
    print(f"✅ Retrieved: {len(retrieval['chunks'])} chunks")
    
    if retrieval['chunks']:
        context = retriever.format_context(retrieval['chunks'])
        answer = generate_answer(test_query, context, lm, check_confidence=True)
        print(f"✅ Answer generated with confidence: {answer['confidence']}")
        print(f"\n📄 Answer: {answer['answer'][:200]}...")

## Step 6: Keep Server Running

In [ ]:
import time

print("⏱️ Keeping server alive... (Press Stop to end)")
print("💬 Open the ngrok URL above in a new tab to access Chainlit")
print("\nServer is running. Do not stop this cell!")

try:
    while True:
        time.sleep(60)
        print(".", end="", flush=True)
except KeyboardInterrupt:
    print("\n\nServer stopped.")